In [5]:
import os
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

In [4]:
def configure():
    load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv('openai_api_key')

In [6]:
def load_documents(file_path):
    if file_path.endswith('.pdf'):
        loader = PyPDFLoader(file_path)
    else:
        loader = TextLoader(file_path)

    documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    docs = text_splitter.split_documents(documents)
    return docs

In [7]:
def create_vector_store(docs):
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(docs, embeddings)
    return vectorstore

In [9]:
def build_rag_pipeline(vectorstore):
    retriever = vectorstore.as_retriever()

    prompt_template = PromptTemplate(
        template="""
        You are a helpful AI assistant. Use the following retrieved context
        to answer the user's query.

        Context: {context}
        Query: {question}

        Provide a detailed and concise response.""", input_variables=['context', 'question']
    )

    llm = ChatOpenAI(model_name="gpt-3.5-turbo-16k", temperature=0.7)

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever = retriever,
        chain_type = 'stuff',
        chain_type_kwargs={'prompt': prompt_template}
    )